In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import matthews_corrcoef
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score, cross_val_predict

C:\Users\User\anaconda3\Lib\site-packages\dask\dataframe\_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


In [2]:
df = pd.read_csv('DOC2VEC.csv')
df.head()

,0,-0.003204968,-0.002140985,-0.002542576,0.001497328,-0.003393872,0.001159331,-0.002889649,-0.000504275,0.002175754,...,0.00368932,-0.002282868,-0.003283782,0.002410355,-0.00189912,0.001885059,0.002705323,-0.002892767,-0.00304901,0.000746865
0,0,0.003493,-0.001401,0.000100,0.003260,-0.002854,-0.000274,0.001107,0.001567,-0.002492,...,0.000811,0.001344,-0.003271,-0.001903,0.001436,0.003367,0.001752,-0.000435,0.001440,-0.003031
1,0,0.003276,-0.000800,-0.002086,0.003244,-0.000005,0.000190,-0.001738,0.001612,-0.000924,...,-0.002907,-0.001561,0.001808,0.000572,-0.001520,0.000899,-0.002558,0.003627,0.001655,0.003378
2,0,-0.002858,0.003405,-0.001726,0.003213,-0.000429,0.001131,0.000291,-0.001390,-0.000128,...,0.001640,-0.001265,0.000430,-0.000098,0.000241,-0.001852,-0.002666,-0.001224,0.000499,0.001196
3,0,0.001331,-0.002792,0.002131,-0.002188,0.002277,-0.001872,0.002020,0.003344,0.002593,...,0.001749,0.002497,0.002995,-0.003240,-0.000091,0.003058,0.001029,-0.001827,0.003049,-0.000808
4,0,-0.001888,0.001836,-0.002530,0.002232,0.002195,-0.002984,-0.000728,0.003184,-0.003780,...,-0.000930,0.003662,-0.000507,-0.001095,-0.002720,0.002565,-0.002618,0.002519,0.001737,-0.000379


In [3]:
y = df['0']
x = df.drop('0', axis=1) 
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.3)

In [4]:
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=42)
x_ros, y_ros = ros.fit_resample(xtrain, ytrain)
total_Metics = []
total_Metics = pd.DataFrame(total_Metics)
total_Metics['Classifier'] = 'Classifier'
total_Metics['Accuracy'] = 'Accuracy'
total_Metics['mcc'] = 'mcc'
# total_Metics['auc'] = 'auc'
total_Metics['Kappa'] = 'Kappa'
total_Metics['precision'] = 'precision'
total_Metics['recall'] = 'recall'
total_Metics['f1'] = 'f1'
total_Metics['sensitivity'] = 'sensitivity'
total_Metics['specificity'] = 'specificity'

models = [RandomForestClassifier(n_estimators = 200, max_depth = 10),
          XGBClassifier(n_estimators = 200,max_depth = 10, learning_rate = 0.1),
          LGBMClassifier(learning_rate = 0.1,max_depth = 10,random_state = 50),
          GradientBoostingClassifier(n_estimators = 200, learning_rate = 0.5, random_state = 50),
          AdaBoostClassifier(n_estimators = 200, learning_rate = 0.1, random_state = 50)]
cv = KFold(n_splits=5, random_state=1, shuffle=True)
for model in models:
  from sklearn.metrics import f1_score, precision_score, recall_score, log_loss, accuracy_score, matthews_corrcoef, roc_auc_score, cohen_kappa_score
  # evaluate model
  # scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
  # model.fit(xtrain, ytrain)
  # pred = model.predict(xtest)
  pred = cross_val_predict(model, x_ros, y_ros, cv=cv, n_jobs=-1)

  # cm1 = confusion_matrix(y, y_pred)
  # report performance
  Accuracy = accuracy_score(y_ros, pred)
  mcc = matthews_corrcoef(y_ros, pred)
  cm1 = confusion_matrix(y_ros, pred)
  kappa = cohen_kappa_score(y_ros, pred)
  f1 = f1_score(y_ros, pred)
  precision_score = precision_score(y_ros, pred)
  recall_score = recall_score(y_ros, pred)
  sensitivity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
  specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
  # y_pred = np.argmax(y_pred, axis=0)
  # auc = roc_auc_score(y, y_pred, multi_class='ovr')
  total_Metics.loc[len(total_Metics.index)] = [model,Accuracy, mcc, kappa, precision_score,recall_score, f1, sensitivity,specificity]

print(total_Metics)

                                          Classifier  Accuracy       mcc  \
0  RandomForestClassifier(max_depth=10, n_estimat...  0.754414  0.517256   
1  XGBClassifier(base_score=None, booster=None, c...  0.721509  0.443888   
2      LGBMClassifier(max_depth=10, random_state=50)  0.718299  0.437008   
3  GradientBoostingClassifier(learning_rate=0.5, ...  0.710273  0.421137   
4  AdaBoostClassifier(learning_rate=0.1, n_estima...  0.617175  0.237885   

      Kappa  precision    recall        f1  sensitivity  specificity  
0  0.508828   0.810176  0.664526  0.730159     0.844302     0.664526  
1  0.443018   0.736301  0.690209  0.712510     0.752809     0.690209  
2  0.436597   0.728188  0.696629  0.712059     0.739968     0.696629  
3  0.420546   0.699695  0.736758  0.717748     0.683788     0.736758  
4  0.234350   0.600000  0.703050  0.647450     0.531300     0.703050  


In [ ]:
1